# Goal:

- provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

- Used as a Python sanity check because I'm not that familiar with R.

In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

np.seterr(all='raise')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Load and Process the Data

In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../data/X_norm_1024.csv"
    y_file = "../data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [3]:
NUM_SAMPLES = 1024
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)

# Same shape as Marcelo's reference code
betas = np.zeros((10, ))

Using subsampled data to compare Python-C++
Reading in ../data/X_norm_1024.csv, ../data/y_1024.csv 
Train X shape is: (1024, 10)
Train y shape is: (1024, 1)


In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas
    return np.expand_dims(sigmoid(preds), -1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]

def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    # for i in tqdm.trange(epochs):
    for i in range(epochs):
        print("\n*******************************************")
        print(f"Iteration: {i}")
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        phi_prime = theta - lr * np.squeeze(gradient)
        if i == 0:
            theta = phi_prime
        else:
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime
        loss = cost(train_x, train_y, theta)
        print(f"Theta: {theta}")
        print(f"Phi: {phi}")
        print(f"Loss: {loss}")
        nesterov_loss[i] = loss

        # print(f"New loss: {cost(train_x, train_y, v)[0]}")
    return nesterov_loss, theta, phi


In [5]:
losses, theta, phi = nesterov(betas, 3, lr, mu, train_x, train_y)


*******************************************
Iteration: 0
Theta: [-1.59405048e-02  4.66008137e-03 -1.81152344e-02  1.51367188e-02
 -1.57226563e-02  1.57828283e-05 -3.32031250e-03  5.85937500e-04
  9.76562500e-04  0.00000000e+00]
Phi: [-1.59405048e-02  4.66008137e-03 -1.81152344e-02  1.51367188e-02
 -1.57226563e-02  1.57828283e-05 -3.32031250e-03  5.85937500e-04
  9.76562500e-04  0.00000000e+00]
Loss: 0.6824088332753198

*******************************************
Iteration: 1
Theta: [-0.03315335  0.00921142 -0.03757909  0.03103008 -0.03228831 -0.00038253
 -0.00648607  0.00066321  0.0017891   0.00056163]
Phi: [-0.03158855  0.00879766 -0.03580965  0.02958523 -0.03078234 -0.00034632
 -0.00619827  0.00065618  0.00171524  0.00051057]
Loss: 0.6713903103516063

*******************************************
Iteration: 2
Theta: [-0.05017535  0.01325299 -0.05672936  0.04628827 -0.0482638  -0.00117161
 -0.00921444  0.00020309  0.00237382  0.00166007]
Phi: [-0.04848564  0.01284796 -0.05482757  0.044

In [6]:
losses[:10]

[0.6824088332753198, 0.6713903103516063, 0.6610257231078792]